Generate pages

## Read the XBRL-instance

In [1]:
from arelle import ViewFileDTS, ViewFileFactTable, ViewFileFactList, ModelManager, FileSource, ViewFileRelationshipSet
from arelle import Cntlr, ModelXbrl, ModelDocument, ViewFileFormulae
from arelle import ModelFormulaObject, RenderingEvaluator, XbrlConst
from arelle.ModelDtsObject import ModelRelationship
import arelle
from arelle import ModelObject, XbrlConst, ViewFile
from arelle.ModelDtsObject import ModelRelationship
from arelle.ModelFormulaObject import ModelParameter, ModelVariable, ModelVariableSetAssertion, ModelConsistencyAssertion


In [2]:
import src
import pandas as pd
import re
import os
from os import listdir, walk, makedirs
from os.path import isfile, join, exists
from io import StringIO

In [3]:
# make sure you have a 'arelle' directory in the data_path! (This is where the taxonomy is stored)
XBRL_DATA_PATH    = 'C:\\Users\\wjwil\\20_local_data\\xbrl\\' 
#DATA_PATH    = 'H:\\20_local_data\\xbrl\\' 
XBRL_RESULTS_PATH = 'C:\\Users\\wjwil\\50_results\\xbrl\\'
LANGUAGE     = "en-GB"
# set the location of taxonomy
os.environ['XDG_CONFIG_HOME'] = XBRL_DATA_PATH 

In [4]:
DOCS_PATH ="..\\docs\\"
DATA_PATH ="..\\data\\"
RULES_PATH = "..\\Solvency2-rules\\"

In [5]:
# Now we make a modelmanager
controller = Cntlr.Cntlr()
controller.webCache.workOffline = True

modelmanager = ModelManager.initialize(controller)
modelmanager.defaultLang = LANGUAGE
modelmanager.formulaOptions = ModelFormulaObject.FormulaOptions()
modelmanager.loadCustomTransforms()

In [6]:
# And we read the XBRL instance
XBRL_INSTANCE = 'qrs_240_instance.xbrl'

xbrl_instance = ModelXbrl.load(modelmanager, XBRL_DATA_PATH + XBRL_INSTANCE)

In [7]:
tables = list(xbrl_instance.relationshipSet("Table-rendering").linkRoleUris)
tables.sort()
print(tables[0:5])

['http://eiopa.europa.eu/xbrl/s2md/role/fws/solvency/solvency2/2019-07-15/tab/S.01.01.02.01', 'http://eiopa.europa.eu/xbrl/s2md/role/fws/solvency/solvency2/2019-07-15/tab/S.01.02.01.01', 'http://eiopa.europa.eu/xbrl/s2md/role/fws/solvency/solvency2/2019-07-15/tab/S.02.01.02.01', 'http://eiopa.europa.eu/xbrl/s2md/role/fws/solvency/solvency2/2019-07-15/tab/S.05.01.02.01', 'http://eiopa.europa.eu/xbrl/s2md/role/fws/solvency/solvency2/2019-07-15/tab/S.05.01.02.02']


## Extract from XBRL-instance

In [8]:
from arelle.ViewUtilFormulae import rootFormulaObjects, formulaObjSortKey

In [41]:
allFormulaRelationshipsSet = xbrl_instance.relationshipSet("XBRL-formulae")
varSetFilterRelationshipSet = xbrl_instance.relationshipSet(XbrlConst.variableSetFilter)
assertionSetRelSet = xbrl_instance.relationshipSet(XbrlConst.assertionSet)
consisAsserFormulaRelSet = xbrl_instance.relationshipSet(XbrlConst.consistencyAssertionFormula)

def rootFormulaObjects():
    if allFormulaRelationshipsSet is None or len(allFormulaRelationshipsSet.modelRelationships) == 0:
        return set()
    rootObjects = set(xbrl_instance.modelVariableSets)
    # remove formulae under consistency assertions from root objects
    for modelRel in consisAsserFormulaRelSet.modelRelationships:
        if modelRel.fromModelObject is not None and modelRel.toModelObject is not None:
            rootObjects.add(modelRel.fromModelObject)   # display consis assertion
            rootObjects.discard(modelRel.toModelObject) # remove formula from root objects
    # remove assertions under assertion sets from root objects
    for modelRel in assertionSetRelSet.modelRelationships:
        if isinstance(modelRel.fromModelObject, ModelObject.ModelObject) and isinstance(modelRel.toModelObject, ModelObject.ModelObject):
            rootObjects.add(modelRel.fromModelObject)   # display assertion set
            rootObjects.discard(modelRel.toModelObject) # remove assertion from root objects
    return rootObjects

In [26]:
def view():
    # determine relationships indent depth
    rootObjects = rootFormulaObjects()
    for rootObject in rootObjects:
        treeDepth(rootObject, 1, set())
    for rootObject in sorted(rootObjects, key=formulaObjSortKey):
        viewFormulaObjects(rootObject, None, 0, set())
    for cfQnameArity in sorted(qnameArity
                               for qnameArity in xbrl_instance.modelCustomFunctionSignatures.keys()
                               if isinstance(qnameArity, (tuple,list))):
        cfObject = xbrl_instance.modelCustomFunctionSignatures[cfQnameArity]
        viewFormulaObjects(cfObject, None, 0, set())

def treeDepth(fromObject, indent, visited):
    if fromObject is None:
        return
    if fromObject not in visited:
        visited.add(fromObject)
        relationshipArcsShown = set()
        for relationshipSet in (varSetFilterRelationshipSet,
                                allFormulaRelationshipsSet):
            for modelRel in relationshipSet.fromModelObject(fromObject):
                if modelRel.arcElement not in relationshipArcsShown:
                    relationshipArcsShown.add(modelRel.arcElement)
                    toObject = modelRel.toModelObject
                    treeDepth(toObject, indent + 1, visited)
        visited.remove(fromObject)

formulae = []
        
def viewFormulaObjects(fromObject, fromRel, indent, visited):
    if fromObject is None:
        return None
    if isinstance(fromObject, (ModelVariable, ModelParameter)) and fromRel is not None:
        text = "{0} ${1}".format(fromObject.localName, fromRel.variableQname)
        xmlRowEltAttr = {"type": str(fromObject.localName), "name": str(fromRel.variableQname)}
    elif isinstance(fromObject, (ModelVariableSetAssertion, ModelConsistencyAssertion)):
        text = "{0} {1}".format(fromObject.localName, fromObject.id)
        xmlRowEltAttr = {"type": str(fromObject.localName), "id": str(fromObject.id)}
    else:
        text = fromObject.localName
        xmlRowEltAttr = {"type": str(fromObject.localName)}
    cols = [text, fromObject.xlinkLabel] # label
    if fromRel is not None and fromRel.elementQname == XbrlConst.qnVariableFilterArc:
        cols.append("true" if fromRel.isCovered else "false") # cover
        cols.append("true" if fromRel.isComplemented else "false") #complement
    else:
        cols.append(None) # cover
        cols.append(None) # compelement
    if isinstance(fromObject, ModelVariable):
        cols.append(fromObject.bindAsSequence) # bind as sequence
    else:
        cols.append(None) # bind as sequence
    if hasattr(fromObject, "viewExpression"):
        cols.append(fromObject.viewExpression) # expression
    else:            
        cols.append(None) # expression
    formulae.append(cols)
    if fromObject not in visited:
        visited.add(fromObject)
        relationshipArcsShown = set()
        for relationshipSet in (varSetFilterRelationshipSet,
                                allFormulaRelationshipsSet):
            for modelRel in relationshipSet.fromModelObject(fromObject):
                if modelRel.arcElement not in relationshipArcsShown:
                    relationshipArcsShown.add(modelRel.arcElement)
                    toObject = modelRel.toModelObject
                    viewFormulaObjects(toObject, modelRel, indent + 1, visited)
        visited.remove(fromObject)


In [27]:
view()

In [39]:
assertions = []
for item in formulae:
    if 'valueAssertion' in item[0]:
        assertions.append(item[1])

d = {}
for assertion in assertions:
    for item in formulae:
        if 'message' in item[0]:
            if assertion == item[1]:
                d[assertion]=item[5]

In [40]:
d

{'s2md_BV1014-2': 'BV1014-2: {{S.08.01.01.01, c0150}} gt 0',
 's2md_BV1015-5': 'BV1015-5: empty({{S.06.02.01.02,c0390}}) where not(matches({{S.06.02.01.02,c0290}},"^..((1.)|(2.)|(5.)|(6.)|(74)|(79)|(8.))$"))',
 's2md_BV1018-2': 'BV1018-2: {{S.08.02.01.01, c0150}} gt 0',
 's2md_BV1019-2': 'BV1019-2: {{S.08.02.01.01, c0170}} gt 0',
 's2md_BV1020-2': 'BV1020-2: {{S.08.02.01.01, c0180}} gt 0',
 's2md_BV1021-2': 'BV1021-2: {{S.08.02.01.01, c0190}} ge 0',
 's2md_BV1022-2': 'BV1022-2: {{S.08.02.01.01, c0200}} ge 0',
 's2md_BV1023-2': 'BV1023-2: {{S.08.02.01.01, c0210}} ge 0',
 's2md_BV1024-2': 'BV1024-2: {{S.08.01.01.01, c0160}} gt 0',
 's2md_BV1025-2': 'BV1025-2: {{S.08.01.01.01, c0170}} gt 0',
 's2md_BV1026-2': 'BV1026-2: {{S.08.01.01.01, c0180}} gt 0',
 's2md_BV1027-2': 'BV1027-2: {{S.08.01.01.01, c0190}} ge 0',
 's2md_BV1028-2': 'BV1028-2: {{S.08.01.01.01, c0200}} ge 0',
 's2md_BV1029-2': 'BV1029-2: {{S.08.01.01.01, c0210}} ge 0',
 's2md_BV1069-4': 'BV1069-4: not(empty({{S.06.02.01.02,c02